In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
import surprise
from surprise.model_selection import cross_validate

import warnings; warnings.simplefilter('ignore')

In [2]:
md = pd. read_csv('C:/Users/varsut/Desktop/Project/Datasets/Input/movies_metadata.csv')
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

## Content Based Recommender
 Content Based Recommenders based on:
* Movie Overviews and Taglines
* Movie Cast, Crew, Keywords and Genre


In [10]:
links_small = pd.read_csv('C:/Users/varsut/Desktop/Project/Datasets/Input/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [109]:
md = md.drop([19730, 29503, 35587])

In [12]:
md['id'] = md['id'].astype('int')

In [13]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 24)

### Description based recommendation

In [14]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [15]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [16]:

tfidf_matrix.shape

(9099, 268124)

In [17]:
#Cosine similarity calculation off all the movies pairwise
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [18]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [19]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [20]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [21]:
get_recommendations('Toy Story').head(10)

2502               Toy Story 2
7535               Toy Story 3
6193    The 40 Year Old Virgin
2547           Man on the Moon
6627              Factory Girl
4702    What's Up, Tiger Lily?
889      Rebel Without a Cause
6554    For Your Consideration
4988          Rivers and Tides
1599                 Condorman
Name: title, dtype: object

### Metadata Based Recommender


In [22]:
credits = pd.read_csv('C:/Users/varsut/Desktop/Project/Datasets/Input/credits.csv')
keywords = pd.read_csv('C:/Users/varsut/Desktop/Project/Datasets/Input/keywords.csv')

In [23]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [24]:
md.shape

(45463, 24)

In [25]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [26]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 27)

In [27]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [28]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [29]:
smd['director'] = smd['crew'].apply(get_director)

In [30]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [31]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [32]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [33]:
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x])

In [34]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [35]:
s = s.value_counts()


In [36]:
s = s[s > 1]

In [37]:
stemmer = SnowballStemmer('english')


In [38]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [39]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [40]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [41]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [42]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [43]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [44]:
get_recommendations('The Dark Knight').head(10)

8031                 The Dark Knight Rises
6218                         Batman Begins
7659            Batman: Under the Red Hood
6623                          The Prestige
1134                        Batman Returns
8927               Kidnapping Mr. Heineken
5943                              Thursday
1260                        Batman & Robin
2085                             Following
9024    Batman v Superman: Dawn of Justice
Name: title, dtype: object

In [45]:
get_recommendations('Toy Story').head(10)

2522                                    Toy Story 2
6386                                       Luxo Jr.
8519                           Toy Story of Terror!
7914                                         Cars 2
6496                                           Cars
1883                                   A Bug's Life
2751                              Creature Comforts
7629                                    Toy Story 3
1432                               Meet the Deedles
4341    The Looney, Looney, Looney Bugs Bunny Movie
Name: title, dtype: object

In [46]:
get_recommendations('Pulp Fiction').head(10)

1381         Jackie Brown
8905    The Hateful Eight
5200    Kill Bill: Vol. 2
4595                Basic
4764             S.W.A.T.
898        Reservoir Dogs
6939              Cleaner
4903    Kill Bill: Vol. 1
231         Kiss of Death
4306       The 51st State
Name: title, dtype: object

#### Popularity and Ratings

Include Popularity and ratings to improve basic recommender

In [47]:
vote_counts = smd[smd['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = smd[smd['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
m = vote_counts.quantile(0.95)
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)


In [48]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [49]:
smd['year'] = pd.to_datetime(smd['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [50]:
improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,year,wr
6623,The Prestige,4510,8,2006,7.344034
8031,The Dark Knight Rises,9263,7,2012,6.801517
6218,Batman Begins,7511,7,2005,6.765201
7659,Batman: Under the Red Hood,459,7,2010,6.108968
2085,Following,363,7,1998,6.073726
1134,Batman Returns,1706,6,1992,5.951183
7561,Harry Brown,351,6,2009,5.923794
8026,Bullet to the Head,490,5,2013,5.736101
9024,Batman v Superman: Dawn of Justice,7189,5,2016,5.203189
1260,Batman & Robin,1447,4,1997,5.123362


In [51]:
improved_recommendations('Toy Story')

,title,vote_count,vote_average,year,wr
3833,"Monsters, Inc.",6150,7,2001,6.726297
7629,Toy Story 3,4710,7,2010,6.668116
2522,Toy Story 2,3914,7,1999,6.623925
8595,The Lego Movie,3127,7,2014,6.566911
6496,Cars,3991,6,2006,5.969604
1883,A Bug's Life,2379,6,1998,5.958577
7404,Cloudy with a Chance of Meatballs,1799,6,2009,5.952358
3016,Chicken Run,1190,6,2000,5.943442
6534,Monster House,912,6,2006,5.938159
7914,Cars 2,2088,5,2011,5.452808


### Collaborative Filtering
Using SVD

In [52]:
reader = surprise.Reader()

In [54]:
ratings = pd.read_csv('C:/Users/varsut/Desktop/Project/Datasets/Input/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [55]:
data = surprise.Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
data.split(n_folds=10)

In [88]:
svd = surprise.SVD()


In [89]:

trainset2 = data.build_full_trainset()

In [90]:
svd.train(trainset2)

In [91]:
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8948  0.8976  0.8945  0.8974  0.8965  0.8962  0.0013  
MAE (testset)     0.6877  0.6905  0.6879  0.6882  0.6894  0.6887  0.0011  
Fit time          8.80    9.83    10.50   9.06    10.36   9.71    0.68    
Test time         0.26    0.31    0.26    0.42    0.28    0.31    0.06    


{'test_rmse': array([0.89481288, 0.89761562, 0.89447013, 0.89744744, 0.89645185]),
 'test_mae': array([0.68771161, 0.69048751, 0.68788256, 0.6881897 , 0.68936691]),
 'fit_time': (8.79590392112732,
  9.829878568649292,
  10.50315809249878,
  9.058931112289429,
  10.363876581192017),
 'test_time': (0.25898313522338867,
  0.30504608154296875,
  0.2599959373474121,
  0.42194509506225586,
  0.2839963436126709)}

In [104]:
ratings[ratings["userId"] == 5]

,userId,movieId,rating,timestamp
351,5,3,4.0,1163374957
352,5,39,4.0,1163374952
353,5,104,4.0,1163374639
354,5,141,4.0,1163374242
355,5,150,4.0,1163374404
356,5,231,3.5,1163373762
357,5,277,4.5,1163373208
358,5,344,3.5,1163373636
359,5,356,4.0,1163374152
360,5,364,4.0,1163373752


In [93]:
svd.predict(500,62718,4.5)

Prediction(uid=500, iid=62718, r_ui=4.5, est=3.5720355897908806, details={'was_impossible': False})

In [106]:
svd.predict(5,6711,2.5)

Prediction(uid=5, iid=6711, r_ui=2.5, est=3.797059566836948, details={'was_impossible': False})

Let us pick user 5000 and check the ratings s/he has given.

For movie with ID 302, we get an estimated prediction of **2.686**. One startling feature of this recommender system is that it doesn't care what the movie is (or what it contains). It works purely on the basis of an assigned movie ID and tries to predict ratings based on how the other users have predicted the movie.

###  Hybrid Recommender


In [94]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [95]:
id_map = pd.read_csv('C:/Users/varsut/Desktop/Project/Datasets/Input/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')
#id_map = id_map.set_index('tmdbId')

In [96]:
indices_map = id_map.set_index('id')

In [97]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [98]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,year,id,est
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.312953
974,Aliens,3282.0,7.7,1986,679,3.282806
1011,The Terminator,4208.0,7.4,1984,218,3.217034
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,3.019185
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,2.956021
1668,Return from Witch Mountain,38.0,5.6,1978,14822,2.837177
8865,Star Wars: The Force Awakens,7993.0,7.5,2015,140607,2.730585
2014,Fantastic Planet,140.0,7.6,1973,16306,2.717642
7705,Alice in Wonderland,8.0,5.4,1933,25694,2.711985
3060,Sinbad and the Eye of the Tiger,39.0,6.3,1977,11940,2.708826


In [99]:
hybrid(500, 'Avatar')

,title,vote_count,vote_average,year,id,est
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.854890
974,Aliens,3282.0,7.7,1986,679,3.713761
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,3.300315
8419,Man of Steel,6462.0,6.5,2013,49521,3.132880
8865,Star Wars: The Force Awakens,7993.0,7.5,2015,140607,3.115909
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,3.086360
2834,Predator,2129.0,7.3,1987,106,3.076686
922,The Abyss,822.0,7.1,1989,2756,3.028837
1011,The Terminator,4208.0,7.4,1984,218,3.014747
7705,Alice in Wonderland,8.0,5.4,1933,25694,2.965202


In [100]:
nmf = surprise.NMF()
cross_validate(nmf, data, measures=['RMSE', 'MAE'],cv=5,verbose= True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9384  0.9430  0.9589  0.9451  0.9426  0.9456  0.0070  
MAE (testset)     0.7200  0.7256  0.7370  0.7287  0.7260  0.7274  0.0055  
Fit time          9.91    10.72   10.21   9.69    10.41   10.19   0.36    
Test time         0.41    0.21    0.21    0.39    0.20    0.29    0.09    


{'test_rmse': array([0.93835309, 0.94299766, 0.958938  , 0.94512529, 0.94263912]),
 'test_mae': array([0.71998087, 0.72558811, 0.73695577, 0.72869281, 0.72602784]),
 'fit_time': (9.9088773727417,
  10.722861051559448,
  10.210912466049194,
  9.694873571395874,
  10.412868022918701),
 'test_time': (0.4089643955230713,
  0.20599746704101562,
  0.2149639129638672,
  0.39496779441833496,
  0.20494580268859863)}

In [102]:
nmf.train(trainset2)

In [103]:
nmf.predict(500,62718,4.5)

Prediction(uid=500, iid=62718, r_ui=4.5, est=4.478265730155721, details={'was_impossible': False})

In [105]:
nmf.predict(5,6711,2.5)

Prediction(uid=5, iid=6711, r_ui=2.5, est=3.6869828186645384, details={'was_impossible': False})

In [108]:
benchmark = []
# Iterate over all algorithms
for algorithm in [surprise.SVD(), surprise.SVDpp(), surprise.SlopeOne(), surprise.NMF(), surprise.NormalPredictor(), surprise.KNNBaseline(), surprise.KNNBasic(), surprise.KNNWithMeans(), surprise.KNNWithZScore(), surprise.BaselineOnly(), surprise.CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=["RMSE","MAE"], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')    


Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


,test_rmse,test_mae,fit_time,test_time
Algorithm,,,,
SVDpp,0.894635,0.686436,20132.943781,19.679142
BaselineOnly,0.896753,0.693440,0.343873,0.424400
SVD,0.902805,0.695691,8.200002,0.659384
KNNBaseline,0.904828,0.692827,0.608576,6.641893
KNNWithZScore,0.924544,0.703286,0.469138,6.272863
KNNWithMeans,0.927282,0.709194,0.375668,5.471668
SlopeOne,0.939761,0.720297,6.457721,17.079120
NMF,0.960794,0.738484,9.717668,0.483666
CoClustering,0.976148,0.755729,4.138232,0.510664
